<a href="https://colab.research.google.com/github/Livia711/livia/blob/main/vexpenses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importando as bibliotecas necessárias
import sqlite3
import pandas as pd

# Conectando ao banco de dados (em memória)
conn = sqlite3.connect(':memory:')

# Criando um cursor para executar as queries
cursor = conn.cursor()

# 1. Criação das tabelas
cursor.execute("""
CREATE TABLE clientes (
  id INTEGER PRIMARY KEY,
  nome TEXT,
  data_criacao DATE
);
""")

cursor.execute("""
CREATE TABLE pedidos (
  id INTEGER PRIMARY KEY,
  cliente_id INTEGER,
  data_pedido DATE,
  valor_total REAL,
  FOREIGN KEY (cliente_id) REFERENCES clientes(id)
);
""")

cursor.execute("""
CREATE TABLE itens_pedido (
  id INTEGER PRIMARY KEY,
  pedido_id INTEGER,
  produto TEXT,
  quantidade INTEGER,
  preco_unitario REAL,
  FOREIGN KEY (pedido_id) REFERENCES pedidos(id)
);
""")

# 2. Inserção de dados fictícios
cursor.execute("""
INSERT INTO clientes (id, nome, data_criacao) VALUES
(1, 'Lívia', '2024-01-10'),
(2, 'Eduardo', '2024-02-15'),
(3, 'Ana', '2024-03-05'),
(4, 'Carlos', '2024-03-15'),
(5, 'Mariana', '2024-04-01');
""")

# Inserindo pedidos com datas distribuídas entre os últimos 12 meses e valores fictícios
cursor.execute("""
INSERT INTO pedidos (id, cliente_id, data_pedido, valor_total) VALUES
(1, 1, '2024-06-15', 200.00),
(2, 1, '2024-07-10', 150.00),
(3, 2, '2024-08-05', 300.00),
(4, 3, '2024-09-20', 180.00),
(5, 3, '2024-10-01', 220.00),
(6, 2, '2024-11-01', 90.00),
(7, 4, '2024-12-25', 110.00),
(8, 5, '2025-01-07', 130.00),
(9, 1, '2025-02-10', 250.00),
(10, 2, '2025-03-01', 180.00),
(11, 3, '2025-04-01', 200.00),
(12, 4, '2025-05-01', 150.00);
""")

# Inserindo itens de pedido
cursor.execute("""
INSERT INTO itens_pedido (id, pedido_id, produto, quantidade, preco_unitario) VALUES
(1, 1, 'Mouse', 2, 50.00),
(2, 1, 'Teclado', 1, 100.00),
(3, 2, 'Monitor', 1, 150.00),
(4, 3, 'Mouse', 1, 50.00),
(5, 3, 'Cabo HDMI', 2, 40.00),
(6, 4, 'Webcam', 2, 90.00),
(7, 5, 'Teclado', 2, 110.00),
(8, 6, 'Cabo HDMI', 3, 30.00),
(9, 7, 'Fone de Ouvido', 1, 50.00),
(10, 8, 'Monitor', 1, 150.00),
(11, 9, 'Teclado', 1, 130.00),
(12, 10, 'Mouse', 2, 50.00),
(13, 11, 'Monitor', 1, 150.00),
(14, 12, 'Fone de Ouvido', 1, 50.00);
""")

# 3. Query: Total de pedidos por cliente
query_1 = """
SELECT
  c.id AS cliente_id,
  c.nome,
  COUNT(p.id) AS total_pedidos
FROM clientes c
LEFT JOIN pedidos p ON c.id = p.cliente_id
GROUP BY c.id, c.nome;
"""
df_total_pedidos = pd.read_sql_query(query_1, conn)

# 4. Query: Ticket médio por cliente
query_2 = """
SELECT
  c.id AS cliente_id,
  c.nome,
  ROUND(SUM(p.valor_total) * 1.0 / COUNT(p.id), 2) AS ticket_medio
FROM clientes c
JOIN pedidos p ON c.id = p.cliente_id
GROUP BY c.id, c.nome;
"""
df_ticket_medio = pd.read_sql_query(query_2, conn)

# 5. Query: Top 5 produtos mais vendidos no último mês
query_3 = """
SELECT
  i.produto,
  SUM(i.quantidade) AS total_vendido
FROM itens_pedido i
JOIN pedidos p ON i.pedido_id = p.id
WHERE strftime('%Y-%m', p.data_pedido) = strftime('%Y-%m', date('now', '-1 month'))
GROUP BY i.produto
ORDER BY total_vendido DESC
LIMIT 5;
"""
df_top_produtos = pd.read_sql_query(query_3, conn)

# 6. Query: Valor total de vendas por mês (últimos 12 meses)
query_4 = """
SELECT
  strftime('%Y-%m', p.data_pedido) AS mes,
  SUM(p.valor_total) AS total_vendas
FROM pedidos p
WHERE p.data_pedido >= date('now', '-12 months')
GROUP BY strftime('%Y-%m', p.data_pedido)
ORDER BY mes;
"""
df_vendas_mes = pd.read_sql_query(query_4, conn)

# Exibindo os resultados
print("Total de Pedidos por Cliente:")
print(df_total_pedidos)

print("\nTicket Médio por Cliente:")
print(df_ticket_medio)

print("\nTop 5 Produtos Mais Vendidos no Último Mês:")
print(df_top_produtos)

print("\nValor Total de Vendas por Mês (últimos 12 meses):")
print(df_vendas_mes)

# Fechando a conexão com o banco de dados
conn.close()

Total de Pedidos por Cliente:
   cliente_id     nome  total_pedidos
0           1    Lívia              3
1           2  Eduardo              3
2           3      Ana              3
3           4   Carlos              2
4           5  Mariana              1

Ticket Médio por Cliente:
   cliente_id     nome  ticket_medio
0           1    Lívia         200.0
1           2  Eduardo         190.0
2           3      Ana         200.0
3           4   Carlos         130.0
4           5  Mariana         130.0

Top 5 Produtos Mais Vendidos no Último Mês:
   produto  total_vendido
0  Monitor              1

Valor Total de Vendas por Mês (últimos 12 meses):
        mes  total_vendas
0   2024-06         200.0
1   2024-07         150.0
2   2024-08         300.0
3   2024-09         180.0
4   2024-10         220.0
5   2024-11          90.0
6   2024-12         110.0
7   2025-01         130.0
8   2025-02         250.0
9   2025-03         180.0
10  2025-04         200.0
11  2025-05         150.0
